In [1]:
!python3 -m pip install --upgrade pip

In [2]:
# !pip install openai==0.27.8
!pip install openai==1.2.4

In [3]:
!pip install python-dotenv tiktoken

In [4]:
!pip install pdfplumber

In [5]:
import openai
import pdfplumber
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv
import os
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
client = OpenAI()

In [7]:
# ファイルをOpenAIのサーバにアップロード
file = client.files.create(
    file=open("./data/栄養・食生活.pdf", "rb"),
    purpose="assistants"
)

In [8]:
file

FileObject(id='file-BtI9w6zbfcK3qN5nwyrvz05c', bytes=543611, created_at=1700315515, filename='栄養・食生活.pdf', object='file', purpose='assistants', status='processed', status_details=None)

- Refarence: [Supprorted files](https://platform.openai.com/docs/assistants/tools/supported-files)

In [9]:
# アシスタントの作成
assistant = client.beta.assistants.create(
    instructions="あなたはQAチャットボットです。 ナレッジベースを使用して、ユーザーの質問に最大限に対応します。",
    model="gpt-4-1106-preview",
    tools=[{"type": "retrieval"}],
    file_ids=[file.id]
)

In [10]:
# 会話するためスレッドの準備
thread = client.beta.threads.create()

In [11]:
# ユーザーメッセージの追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="毎日の必要なカルシウムの摂取量は？"
)

In [12]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)

for message in messages:
    print(message.role, ":", message.content[0].text.value)

user : 毎日の必要なカルシウムの摂取量は？


In [13]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_6cB5Y726vAYhthkFXeNA4o2t', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='毎日の必要なカルシウムの摂取量は？'), type='text')], created_at=1700315518, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_iOBhMeAub146O9Ntwgy5oMA7')], object='list', first_id='msg_6cB5Y726vAYhthkFXeNA4o2t', last_id='msg_6cB5Y726vAYhthkFXeNA4o2t', has_more=False)

In [25]:
# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [26]:
# 実行状況の確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

in_progress


In [27]:
# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)
for message in messages:
    print(message.role, ":", message.content[0].text.value)

user : 毎日の必要なカルシウムの摂取量は？
assistant : 毎日の必要なカルシウムの摂取量について、成人で600～700mgの摂取量が必要とされています。現在の資料では、確定的な情報を引用することが技術的な問題によりできませんが、提供された資料によれば、成人の平均摂取量は571mgであり、目標値の範囲には若干足らない状況であると指摘されています。


In [28]:
# メッセージのannotationsの抽出
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# 注釈の追加
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f' [{index}]')

    if (file_citation := getattr(annotation, 'file_citation', None)):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
    elif (file_path := getattr(annotation, 'file_path', None)):
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
        # 注: 簡潔にするため、ファイルのダウンロードは上記では実装していません

# ユーザーに表示する前にメッセージの末尾に脚注を追加
message_content.value += '\n' + '\n'.join(citations)

In [29]:
message_content

Text(annotations=[], value='毎日の必要なカルシウムの摂取量について、成人で600～700mgの摂取量が必要とされています。現在の資料では、確定的な情報を引用することが技術的な問題によりできませんが、提供された資料によれば、成人の平均摂取量は571mgであり、目標値の範囲には若干足らない状況であると指摘されています。\n')

In [30]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_6cB5Y726vAYhthkFXeNA4o2t', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='毎日の必要なカルシウムの摂取量は？'), type='text')], created_at=1700315518, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_iOBhMeAub146O9Ntwgy5oMA7'), ThreadMessage(id='msg_7CakLCcJsgR7zQY4kiahplHE', assistant_id='asst_X0o8YNTTDmf33OOxLHvHrids', content=[MessageContentText(text=Text(annotations=[], value='毎日の必要なカルシウムの摂取量について、成人で600～700mgの摂取量が必要とされています。現在の資料では、確定的な情報を引用することが技術的な問題によりできませんが、提供された資料によれば、成人の平均摂取量は571mgであり、目標値の範囲には若干足らない状況であると指摘されています。\n'), type='text')], created_at=1700315528, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_afjAEklFlGGPl7gqyITCmbCE', thread_id='thread_iOBhMeAub146O9Ntwgy5oMA7')], object='list', first_id='msg_6cB5Y726vAYhthkFXeNA4o2t', last_id='msg_7CakLCcJsgR7zQY4kiahplHE', has_more=False)

In [31]:
# ユーザーメッセージの追加
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="今日の天気は？"
)

# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)

# アシスタントにリクエスト
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# 実行状況の確認
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(run.status)

# スレッドのメッセージリストの確認
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    order="asc"
)

for message in messages:
    print(message.role, ":", message.content[0].text.value)

in_progress
user : 毎日の必要なカルシウムの摂取量は？
assistant : 毎日の必要なカルシウムの摂取量について、成人で600～700mgの摂取量が必要とされています。現在の資料では、確定的な情報を引用することが技術的な問題によりできませんが、提供された資料によれば、成人の平均摂取量は571mgであり、目標値の範囲には若干足らない状況であると指摘されています。
assistant : I apologize for the confusion. There was an error in extracting the exact quote from the document. The necessary daily intake of calcium for adults is stated to be between 600–700 mg. However, the average intake for adults is reported to be 571 mg, which suggests that there is room for improvement to meet the recommended intake levels. Please note that these figures can vary based on individual health requirements, life stage, and dietary guidelines in different countries or regions. Always consult with a healthcare professional for personalized advice.
user : 今日の天気は？
assistant : 
